In [129]:
import pandas as pd
import numpy as np

import datetime as dt

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.utils import resample

import pickle


import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)   
    
from src.sports_scrapers import scrape_huskies, scrape_seahawks
from src.weather_scraper import get_raw_forecast, get_raw_forecast_dataframe, get_hi_temperature, seattle_weather_fcst
from src.data_retrievers import DataRetrieval
from src.holiday_calendars import SeattleHolidays
from src.featurizers import (CountCalls, FeaturizeCalls, DateDummies, HolidayDummies, 
                             EventDummies, MakeDummies, FeaturizeDates, JoinDataFrames,
                             MakeModelInput, AddWeatherForecast)
from src.models import calls_pipe, baseline_model, city_model, neighborhood_dist_model, model_ensemble

import folium
from folium import GeoJson
import shapefile
import branca.colormap as cm
import json

plt.style.use('ggplot')
import matplotlib.pyplot as plt
%matplotlib inline

# Get initial calls for service data

In [40]:
retriever = DataRetrieval()

In [41]:
calls = retriever.get_calls_data()

In [42]:
targets, features = calls_pipe(calls)

/Users/analyticallycorrect/galvanize/capstone/seattle_domestic_violence/src/data_retrievers.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["date"] = ""
/Users/analyticallycorrect/galvanize/capstone/seattle_domestic_violence/src/data_retrievers.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["seahawks_game"][i] = "home_Playoffs"
/Users/analyticallycorrect/galvanize/capstone/seattle_domestic_violence/src/data_retrievers.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#  Train-Test Split

In [43]:
X_train, X_test, y_train, y_test =  train_test_split(features, targets,
                                                     test_size=0.2, random_state=157)


# Baseline Modle

In [57]:
neighborhood_model = baseline_model(X_train, y_train)

In [58]:
neighborhood_train_predictions = neighborhood_model.predict(X_train)
neighborhood_train_mse = mean_squared_error(y_train, neighborhood_train_predictions)

neighborhood_test_predictions = neighborhood_model_b.predict(X_test)
neighborhood_test_mse = mean_squared_error(y_test, neighborhood_test_predictions)

In [59]:
neighborhood_train_mse, neighborhood_test_mse

(0.501661048512353, 0.6271119612526889)

In [60]:
neighborhood_model.score(X_train, y_train)

0.11125025868723953

# City Model - GBRT

In [63]:
city_model = city_model(X_train, y_train)

In [66]:
train_pred_city = model_city.predict(X_train)
test_pred_city = model_city.predict(X_test)

In [65]:
mean_squared_error(y_train.sum(axis=1), model_city.predict(X_train)), mean_squared_error(y_test.sum(axis=1), model_city.predict(X_test))

(29.71786482471047, 34.05285266499792)

In [64]:
model_city.score(X_train, y_train.sum(axis=1))

0.28958052946245827

# Neighborhood Distribution - Random Forest


In [69]:
rf_dist = neighborhood_dist_model(X_train, y_train)

In [68]:
neighborhood_dist_train = pd.DataFrame(np.array(y_train.T) / np.array(y_train.sum(axis=1))).T
neighborhood_dist_test = pd.DataFrame(np.array(y_test.T) / np.array(y_test.sum(axis=1))).T

In [70]:
train_pred_dist = rf_dist.predict(X_train)
test_pred_dist = rf_dist.predict(X_test)

mean_squared_error(neighborhood_dist_train, train_pred_dist), mean_squared_error(neighborhood_dist_test, test_pred_dist)

(0.0005990147695879705, 0.0006336926274971152)

In [72]:
rf_dist.score(X_train, neighborhood_dist_train)

0.04982028605339261

# Create Model Ensemble

In [74]:
train_pred_comb = model_ensemble(train_pred_city, neighborhood_dist_train)
test_pred_comb = model_ensemble(test_pred_city, neighborhood_dist_test)

In [75]:

mean_squared_error(y_train, train_pred_comb.T), mean_squared_error(y_test, test_pred_comb.T)

(0.032882601550400306, 0.03728514228671553)

# Create Forecast

In [78]:
weather_avg = pd.read_csv('../data/weather_averages.csv')

weather_fcst = weather_avg[['DATE', 'DLY-TMAX-NORMAL', 'DLY-PRCP-50PCTL', 'DLY-SNOW-50PCTL']]

weather_fcst['DATE'] = pd.to_datetime(weather_fcst['DATE'].astype('str'), format='%Y%m%d', errors='ignore')

weather_fcst['month_day'] = weather_fcst['DATE'].dt.strftime('%m/%d')

weather_fcst = (weather_fcst[['month_day', 'DLY-PRCP-50PCTL', 'DLY-TMAX-NORMAL', 'DLY-SNOW-50PCTL']]
                .rename(columns={'DLY-PRCP-50PCTL':'precip', 'DLY-TMAX-NORMAL':'temp_max',
                                 'DLY-SNOW-50PCTL':'snow'}))
weather_fcst['snow'] = 0.0 

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [79]:
start_date = '12/03/2018'
end_date = '12/31/2019'
model_end = ('09/30/2018', 3194)

In [84]:
forecast_pipe = Pipeline(steps=[
    ('date_featurizer', FeaturizeDates(start_date, end_date, model_end)),
    ('date_dummifier', DateDummies()),
    ('model_input', MakeModelInput()),
    ('add_weather', AddWeatherForecast()),
])

In [86]:
forecast_pipe.fit(None)

Pipeline(memory=None,
     steps=[('date_featurizer', <src.featurizers.FeaturizeDates object at 0x1a29f34cc0>), ('date_dummifier', <src.featurizers.DateDummies object at 0x1a29f34cf8>), ('model_input', <src.featurizers.MakeModelInput object at 0x1a29f34b38>)])

In [119]:
forecast_features = forecast_pipe.transform(None)

/Users/analyticallycorrect/galvanize/capstone/seattle_domestic_violence/src/data_retrievers.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["date"] = ""
/Users/analyticallycorrect/galvanize/capstone/seattle_domestic_violence/src/data_retrievers.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["seahawks_game"][i] = "home_Playoffs"
/Users/analyticallycorrect/galvanize/capstone/seattle_domestic_violence/src/data_retrievers.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [348]:
forecast_predictions = model_city.predict(forecast_features)

In [349]:
results = pd.DataFrame({'date': forecast_dates['dt_time'], 'forecast_pred':forecast_predictions})

In [356]:
neighborhood_predicted_counts = results['forecast_pred'].values.copy() * neighborhood_predictions.T

In [357]:
neighborhood_predicted_counts = pd.DataFrame(neighborhood_predicted_counts.T, columns=targets.columns)

In [358]:
neighborhood_predicted_counts = neighborhood_predicted_counts.join(results['date'])

In [359]:
neighborhood_predicted_counts.to_csv('../data/neighborhood_predictions.csv', index=False)

In [360]:
neighborhood_means = neighborhood_predicted_counts.mean()

In [361]:
neighborhood_std = neighborhood_predicted_counts.std()

In [462]:
ratings = (((neighborhood_means - neighborhood_predicted_counts.drop('date', axis=1)) / (neighborhood_std*2)) + .5)

In [463]:
ratings.reset_index(inplace=True)

In [466]:
ratings = ratings.join(results['date'])

In [365]:
ratings.to_csv('../data/neighborhood_ratings.csv', index=False)